# Working with Massive Text Data

**TA Session - Text and NLP for Data Science**  
Barcelona School of Economics

---

## The Problem

You have a 16GB CSV of news articles. You need articles from March 2023 only.

Your laptop has 16GB of RAM.

What do you do?

---

## Section 1: The Naive Approach (Watch It Fail)

Let's try what every pandas user does first:

In [1]:
import os
import psutil

# Check our environment
ram_gb = psutil.virtual_memory().total / (1024**3)
print(f"Your machine has {ram_gb:.1f} GB of RAM")

data_file = "../data/cc_news_large.csv"
file_size_gb = os.path.getsize(data_file) / (1024**3)
print(f"The data file is {file_size_gb:.1f} GB")
print()
print(f"Ratio: file is {file_size_gb/ram_gb*100:.0f}% of your RAM")
print("Pandas typically needs 2-5x the file size in memory...")

Your machine has 8.0 GB of RAM
The data file is 0.7 GB

Ratio: file is 9% of your RAM
Pandas typically needs 2-5x the file size in memory...


In [2]:
# ⚠️ WARNING: This cell WILL crash or hang on most machines.
# That's the point. Run it, wait 30-60 seconds, then interrupt the kernel.
#
# Watch your RAM usage (Activity Monitor on Mac, Task Manager on Windows)
# as this runs. You'll see it spike and then... bad things happen.

import pandas as pd

# The naive approach - just load everything
df = pd.read_csv("../data/cc_news_large.csv")

KeyboardInterrupt: 

### What just happened?

If you're lucky: `MemoryError`

If you're unlucky: Your laptop started swapping to disk, became unresponsive, and you had to force-kill Python.

**Why?** Pandas loads the ENTIRE file into memory before you can do anything with it. A 10GB CSV file typically needs 20-30GB of RAM to load into a DataFrame (due to Python object overhead, string storage, index creation, etc.).

Your laptop doesn't have that. So pandas tries to use disk as overflow memory (swapping), which is ~1000x slower than RAM, and everything grinds to a halt.

---

In [3]:
# If for whatever reason you were able to load it, you could filter like this... se how long it takes.
df[df["date"].str.startswith("2023-03")]

NameError: name 'df' is not defined

In [4]:
# Delete the dataframe to free up memory
del df

NameError: name 'df' is not defined

## Section 2: The Old Way - Chunked Reading

Before modern tools existed, we had to do this manually. It's educational to see why.

In [ ]:
import pandas as pd
from datetime import datetime
import time

# The chunked approach: read the file in pieces
CHUNK_SIZE = 50_000  # rows per chunk
TARGET_START = "2023-03-01"
TARGET_END = "2023-04-01"

start_time = time.time()
matching_chunks = []
total_rows_scanned = 0

print(f"Scanning file in chunks of {CHUNK_SIZE:,} rows...")
print(f"Looking for dates between {TARGET_START} and {TARGET_END}")
print()

for i, chunk in enumerate(pd.read_csv("../data/cc_news_large.csv", chunksize=CHUNK_SIZE)):
    total_rows_scanned += len(chunk)
    
    # Filter this chunk for our date range
    mask = (chunk['date'] >= TARGET_START) & (chunk['date'] < TARGET_END)
    matching = chunk[mask]
    
    if len(matching) > 0:
        matching_chunks.append(matching)
    
    # Progress update
    if (i + 1) % 10 == 0:
        elapsed = time.time() - start_time
        print(f"  Chunk {i+1}: scanned {total_rows_scanned:,} rows, "
              f"found {sum(len(c) for c in matching_chunks):,} matches, "
              f"elapsed {elapsed:.1f}s")

# Combine all matching chunks
if matching_chunks:
    result = pd.concat(matching_chunks, ignore_index=True)
else:
    result = pd.DataFrame()

elapsed = time.time() - start_time
print()
print(f"Done! Found {len(result):,} articles from March 2023")
print(f"Total time: {elapsed:.1f} seconds")
print(f"Scanned {total_rows_scanned:,} rows")

In [ ]:
# Let's see what we got
result.head()

In [ ]:
# How much RAM are we using now?
ram_used_gb = psutil.virtual_memory().used / (1024**3)
print(f"RAM used at this point in time: {ram_used_gb:.1f} GB")

### The Chunked Approach: Analysis

**Pros:**
- It works! Doesn't crash.
- Memory stays bounded (only one chunk in RAM at a time)

**Cons:**
- Slow: must scan the entire file even if target data is at the end
- Tedious: you're writing custom iteration logic
- Error-prone: easy to mess up the chunk boundary handling
- Doesn't scale: what if you need to do this 10 times with different filters?

**The deeper problem:** You're essentially writing a half-baked database query engine by hand. Someone already did this work properly. Let's use their tools.

---

## Section 3: The Modern Way

Two tools that solve this properly: **DuckDB** and **Polars**.

### Option A: Polars

Polars is a DataFrame library like pandas, but designed for speed and memory efficiency.

The key feature: **lazy evaluation**. You build up a query plan, and Polars optimizes and executes it only when you call `.collect()`.

In [ ]:
import polars as pl
import time

start_time = time.time()

# scan_csv (not read_csv!) creates a lazy query
result_polars = (
    pl.scan_csv("../data/cc_news_large.csv")
    .filter(pl.col("date") >= "2023-03-01")
    .filter(pl.col("date") < "2023-04-01")
    .collect()  # Execute the query
)

elapsed = time.time() - start_time
print(f"Polars: Found {len(result_polars):,} articles in {elapsed:.1f} seconds")

In [ ]:
result_polars.head()

### Understanding Lazy Evaluation

Watch what happens when you DON'T call `.collect()`:

In [ ]:
# This returns INSTANTLY - no data is read yet!
lazy_query = (
    pl.scan_csv("../data/cc_news_large.csv")
    .filter(pl.col("date") >= "2023-03-01")
    .filter(pl.col("date") < "2023-04-01")
    .select(["date", "title", "domain"])
)

print("Type:", type(lazy_query))
print()
print("Query plan:")
print(lazy_query.explain())

Polars shows you the execution plan. Notice how it pushes the filter down and only reads the columns you selected. This is query optimization - the same thing databases do.

Only when you call `.collect()` does the actual work happen.

> The main difference with read_csv is that this one would get the whole thing into memory to filter, instead of just bringing to memory what its actually needed.

In [ ]:
# More complex Polars example: aggregation with word counting
articles_per_day = (
    pl.scan_csv("../data/cc_news_large.csv")
    .filter(pl.col("date") >= "2023-03-01")
    .filter(pl.col("date") < "2023-04-01")
    .with_columns([
        pl.col("text").str.split(" ").list.len().alias("word_count")
    ])
    .group_by("date")
    .agg([
        pl.len().alias("article_count"),
        pl.col("word_count").mean().alias("avg_words")
    ])
    .sort("date")
    .collect()
)

articles_per_day

### Option B: DuckDB

DuckDB is an embedded analytical database. It's like SQLite, a simple small file-based database system, but designed for data analysis instead of transactions.

The magic: it can run SQL queries directly on CSV/Parquet files without loading them into memory.

In [ ]:
import duckdb
import time

start_time = time.time()

# One line. That's it.
result_duckdb = duckdb.sql("""
    SELECT *
    FROM read_csv_auto('../data/cc_news_large.csv')
    WHERE date >= '2023-03-01' AND date < '2023-04-01'
""").df()

elapsed = time.time() - start_time
print(f"DuckDB: Found {len(result_duckdb):,} articles in {elapsed:.1f} seconds")

In [ ]:
result_duckdb.head()

In [ ]:
type(result_duckdb)

### Why is DuckDB fast?

1. **Streaming execution**: It doesn't load the whole file. It streams through it, filtering as it goes.
2. **Columnar processing**: It can skip columns you don't need entirely.
3. **Vectorized operations**: Processes data in batches, not row-by-row.
4. **Predicate pushdown**: The WHERE clause is applied during reading, not after.

You wrote one SQL query. DuckDB figured out the optimal execution strategy.

### More DuckDB Examples

You can do complex analytics without ever loading the full file:

In [ ]:
# Count articles per domain, across the ENTIRE 16GB file
domain_counts = duckdb.sql("""
    SELECT 
        domain,
        COUNT(*) as article_count,
        AVG(LENGTH(text)) as avg_text_length
    FROM read_csv_auto('../data/cc_news_large.csv')
    GROUP BY domain
    ORDER BY article_count DESC
    LIMIT 20
""").df()

domain_counts

In [ ]:
# Find articles containing specific keywords
inflation_articles = duckdb.sql("""
    SELECT date, title, domain, LENGTH(text) as text_length
    FROM read_csv_auto('../data/cc_news_large.csv')
    WHERE text ILIKE '%inflation%'
    AND date >= '2023-01-01'
    ORDER BY date DESC
    LIMIT 10
""").df()

inflation_articles

---

## Section 4: Comparison Summary

| Approach | Time | Memory | Code Complexity |
|----------|------|--------|----------------|
| Naive pandas | ❌ Crashes | ❌ Exceeds RAM | Simple (but useless) |
| Chunked pandas | Slow | ✓ Bounded | Complex (manual) |
| DuckDB | Fast | ✓ Efficient | Simple (SQL) |
| Polars | Mid | ✓ Efficient | Simple (fluent API) |

### When to use what?

**DuckDB:**
- You know SQL
- One-off analytics queries
- Joining multiple large files
- Working with Parquet files (even faster)

**Polars:**
- You prefer Python-native APIs
- Building data pipelines
- Need pandas-like operations but faster

**Chunked pandas:**
- Legacy code you can't rewrite
- Very specific streaming requirements
- Otherwise: just don't. Use DuckDB or Polars.

---

## Bonus: Converting to Parquet

If you'll work with this data repeatedly, convert it to Parquet format. Parquet is:
- Columnar (only read columns you need)
- Compressed (16GB CSV → ~2GB Parquet)
- Much faster to read

DuckDB can do this conversion without loading the full file into memory:

In [ ]:
# Convert CSV to Parquet (run once, benefit forever)
# It takes a couple of minutes...

duckdb.sql("""
    COPY (SELECT * FROM read_csv_auto('../data/cc_news_large.csv'))
    TO '../data/cc_news_large.parquet' (FORMAT PARQUET, COMPRESSION ZSTD)
""")

In [ ]:
# Then queries on Parquet are even faster (half the time):
duckdb.sql("""
    SELECT *
    FROM '../data/cc_news_large.parquet'
    WHERE date >= '2023-03-01' AND date < '2023-04-01'
""").df()

In [ ]:
duckdb.sql("""
    SELECT date
    FROM '../data/cc_news_large.parquet'
    WHERE date >= '2023-03-01' AND date < '2023-04-01'
""").df()

---

## Key Takeaways

1. **Pandas is not a database.** It loads everything into memory. For files bigger than ~1/3 of your RAM, it will struggle or fail.

2. **The "big data" threshold is lower than you think.** On a 16GB laptop, a 5GB file is already problematic for pandas.

3. **Modern tools make "hard" problems trivial.** DuckDB and Polars handle large files elegantly. Learn them.

4. **Lazy evaluation is a superpower.** Building a query plan before executing means the tool can optimize. You don't have to.

5. **When in doubt, use SQL.** DuckDB lets you write SQL against CSV files. If you know SQL, you already know how to handle big files.

---

## Next: Exercise

Open `02_exercise.ipynb` to apply what you've learned.